In [18]:
# ======================
# 1. IMPORTS
# ======================
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import pickle  # To save models
import os

In [45]:
# ======================
# 2. LOAD DATA
# ======================
df = pd.read_csv('../../raw_data/all_clean_weather_1984_2024_filtered.csv', parse_dates=['DATE'])  # <-- Update your path

# Rename columns
df.rename(columns={
    'TX': 'max_temperature',
    'TM': 'avg_temperature',
    'TN': 'min_temperature',
    'NOM_USUEL': 'location',
    'LAT': 'lat',
    'LON': 'lon',
    'DATE': 'DATE',
    'RR': 'amount_precipitation',
    'Co2 Mole fraction (ppm)': 'co2',
    'Methane ppm': 'ch4'
}, inplace=True)

# First replace ',' by '.' then convert to float
df['co2'] = df['co2'].str.replace(',', '.').astype(float)
df['ch4'] = df['ch4'].str.replace(',', '.').astype(float)

# Extract year and month
df['year'] = df['DATE'].dt.year
df['month'] = df['DATE'].dt.month

In [40]:
# ======================
# 3. TRAIN A PROPHET MODEL PER CITY
# ======================

# Create a folder to save models
os.makedirs('prophet_models', exist_ok=True)

# Get list of cities
cities = df['location'].unique()
for city in cities:
    city_df = df[df['location'] == city]

    # Clean city name for saving (replace / and space and \)
    safe_city = city.replace('/', '_').replace(' ', '_').replace('\\', '_')

    # Monthly aggregation
    monthly_avg = city_df.groupby(['year', 'month']).agg({
        'avg_temperature': 'mean',
        'co2': 'mean',
        'ch4': 'mean',
        'amount_precipitation': 'mean'
    }).reset_index()

    # Prepare Prophet dataframe
    monthly_avg['ds'] = pd.to_datetime(monthly_avg['year'].astype(str) + '-' + monthly_avg['month'].astype(str) + '-01')
    monthly_avg['y'] = monthly_avg['avg_temperature']
    prophet_df = monthly_avg[['ds', 'y', 'co2', 'ch4', 'amount_precipitation']]

    # Train Prophet
    model = Prophet(
        yearly_seasonality=True,
        daily_seasonality=False,
        weekly_seasonality=False,
        seasonality_mode='multiplicative',
    )
    model.add_regressor('co2', prior_scale=2.0)
    model.add_regressor('ch4', prior_scale=1.0)
    model.add_regressor('amount_precipitation', prior_scale=1.0)

    model.fit(prophet_df)

    # Save model using SAFE name
    with open(f'prophet_models/{safe_city}_prophet.pkl', 'wb') as f:
        pickle.dump(model, f)
        
print("✅ All city models trained and saved!")

15:04:28 - cmdstanpy - INFO - Chain [1] start processing
15:04:28 - cmdstanpy - INFO - Chain [1] done processing
15:04:28 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1] done processing
15:04:29 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1] done processing
15:04:29 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1] done processing
15:04:29 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1] done processing
15:04:29 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1] done processing
15:04:29 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1] done processing
15:04:29 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1] done processing
15:04:29 - cmdstanpy - INFO - Chain [1] start processing
15:04:29 - cmdstanpy - INFO - Chain [1]

✅ All city models trained and saved!
